# 機械学習における最適化

この章では、最適化手法、中でもニューラルネットワークの訓練のための連続最適化手法を中心に解説する。

理がない限りニューラルネットワークの構造としてはシンプルな多層パーセプトロン(MLP)を対象とし、損失関数としては回帰問題に対する平均二乗誤差、分類問題に対する交差エントロピー誤差を想定しているが、なるべく一般的な内容を扱うようにする。

また、現代においても新たな最適化手法が提案され続けているため、ここでは代表的な手法を紹介するにとどめる。

## 勾配降下法


勾配降下法(Gradient Descent, GD)は、関数の最小値を見つけるための最も基本的な最適化手法である。

関数$f(\boldsymbol{\theta})$の勾配$\nabla f(\boldsymbol{\theta})$は、関数が最も急激に増加する方向を示すベクトルであり、
勾配降下法ではこの勾配の反対方向にパラメータ$\boldsymbol{\theta}$を更新することで、関数の値を減少させる、というのが基本的な考え方である。

最も単純なものが最急降下法であり、パラメータ$\boldsymbol{\theta}$の更新は以下のように行われる。

$$
W_i := W_i - \eta \frac{\partial L}{\partial W_i}
$$

$L$は目的関数で、$\eta$は学習率(パラメータ更新のスケールを決めるパラメータ)とした。

ANN(MLP)の最適化においてはしばし、訓練データを小さなサブ集合(ミニバッチ)に分割し、各ミニバッチに対して勾配降下法を適用することが行われる。
加えて、ミニバッチの中からランダムにサンプルを選ぶことも多く、これを確率的勾配降下法(Stochastic Gradient Descent, SGD)と呼ぶ。
バッチに分けることで、各ステップでの計算コストが削減される上に、固定のデータを常に用いるよりも、ある種のノイズが導入され、局所最適解から脱出できることがある。

$$
\begin{align*}
W_i & := W_i - \eta \frac{\partial L_{batch}}{\partial W_i} \\
L_{batch} & := \frac{1}{|B|} \sum_{n \in B} (y_n - f(x_n; \boldsymbol{W}))^2
\end{align*} 
$$

バッチサイズ$|B|$はミニバッチのサンプル数である。とくに$|B|=1$の場合をオンライン学習(online learning)と呼んだりもする。

SGDなどの勾配法はシンプルで実装も容易であるが、中々収束しない、局所最適解に陥りやすい、などの問題が生じることもある。加えて学習率をどのように設定するのかが非自明である。
データの特性に応じて、学習をステップに分けて学習率を変更させたり、指数関数や多項式を用いて学習が進むにつれて小さな学習率に変更するなどの工夫もなされてきたが、最適な学習率のスケジュールを見つけるのは一般に難しい。

そうした経緯から提案されたのが、勾配だけでなく、過去の更新情報を利用するモーメンタム法や、パラメータごとに異なる学習率を適応的に調整するAdaGrad, RMSProp, Adamなどの手法である。
年代に沿ってこれらの手法を紹介するのも楽しいが、続く節では、現代においても広く用いられているAdamを中心に説明する。


## Adam

Adamは、勾配降下法の様にその都度の勾配の情報だけを使うのではなく、以前の勾配の情報も有効活用しながら学習率も調整する手法である。

2014年にDiederik P. KingmaとJimmy Baによって提案されて以来、深層学習の分野で現代に至るまで広く用いられている。→[arXiv:1412.6980](https://arxiv.org/abs/1412.6980)

Adamでは、各パラメータ$W_i$に対して、以下のように更新を行う。

$$
\begin{align*}
g_t & = \nabla_{W} L_t \\
m_t & = \beta_1 m_{t-1} + (1 - \beta_1) g_t \\
v_t & = \beta_2 v_{t-1} + (1 - \beta_2) g_t^2 \\
\hat{m}_t & = \frac{m_t}{1 - \beta_1^t} \\
\hat{v}_t & = \frac{v_t}{1 - \beta_2^t} \\
W_t & = W_{t-1} - \eta \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon} 
\end{align*}
$$

なお$m_t, v_t$はそれぞれ1次モーメンタム、2次モーメンタムと呼ばれ、element-wiseに計算する。
$\beta_1, \beta_2$はそれぞれ1次モーメンタム、2次モーメンタムの減衰率を表すハイパーパラメータであり、よく$\beta_1=0.9, \beta_2=0.999$が用いられる。
実際の更新式に用いるハット付きの量は、初期値を0に設定した場合のバイアスを補正するためのものである。

一次モーメント$m_t$は、勾配の指数移動平均を表し、過去の勾配情報を蓄積することで、勾配のノイズを平滑化し、安定した更新を可能にする。
二次モーメント$v_t$は、勾配の二乗の指数移動平均を表し、各パラメータの勾配の大きさを捉える。
これにより、勾配の大きさに応じて学習率を調整し、勾配が大きいパラメータの更新を抑制し、勾配が小さいパラメータの更新を促進する。

### AdamW

AdamWは、Adamの変種であり過学習を防止するための正則化手法であるWeight decay(重み減衰)を組み込んだものである。

SGDにおけるWeight decayは、パラメータの大きさを抑制することで過学習を防止する手法であり、L2正則化と等価になるが、Adamに対して単純にパラメータ更新の式にL2正則化項を加えてしまうと、重み減衰の効果が正しく働かないことが指摘され、そこで提案されたのがAdamWである。

AdamWの論文は、2020年にLoshchilovとHutterによって発表された[arXiv:1711.05101](https://arxiv.org/abs/1711.05101)。

SGDにおけるWeight decayは、パラメータ更新の際の勾配に正則化項の勾配を加えることで実現される。

$$
g_t = \nabla_{W} L_t + \lambda W_{t}
$$

これは、誤差関数$L_t$に対してL2正則化項$\frac{\lambda}{2} ||W||^2$を加えたものを考えて勾配を計算していることに相当する。
この方法とのアナロジーで、Adamにおける更新式に正則化項の勾配を加えることで正則化が期待できるが、Adamの更新式は勾配の大きさに応じて学習率を調整するため、正則化項の効果が不均一になり、期待通りに働かないことがある。

そこで、AdamWでは、Weight decayをパラメータ更新の際に直接適用することで、正則化の効果を均一に保つようにしている。

$$
W_t = W_{t-1} - \eta \left( \frac{\hat{g}_t}{\sqrt{v_t} + \epsilon} + \lambda W_{t-1} \right)
$$

## 二階微分を用いる方法

勾配のみを用いる手法は、関数の形状に関する情報が限られているため、収束が遅い、局所最適解に陥りやすい、などの問題がある。これに対処するために、ヘッセ行列を用いた二階微分の情報を活用する方法がある。
二階微分の情報を用いることで、関数の曲率に関する情報を得ることができ、より効率的な最適化が可能になる。
一方で、ヘッセ行列の計算は計算コストが高く、特に高次元のパラメータ空間では実用的でないことが多い。

代表的な手法として、ニュートン法や準ニュートン法(L-BFGS法, etc.)などがある。
ニュートン法では、ヘッセ行列の逆行列を用いてパラメータを更新するのに対して、
準ニュートン法では、ヘッセ行列の近似を用いることで計算コストを削減する違いがある。


## 残差接続

最適化手法とは異なるが、ニューラルネットワークの構造として残差接続(Residual Connection)が提案されている。



